## Header

### Import Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from binance.client import Client
from datetime import datetime

### Initialise API

In [2]:
API_KEY = 'JPFZLK0ZgvhSniD3fvCNDQSC8UFin0kLKeIOJToe9LyGaJhQG29ZVWqTuZ792jAV'
API_SECRET = 'jw1UUQ4r5xFUgZHSiZcS8hssAKqwDYr8QzehZiNTRnXZBRWvpNQ9mtRyGcS4vfja'

client = Client(API_KEY, API_SECRET)

## Define Wallet Class

In [3]:
class Wallet:
    def __init__(self, initial_balance):
        self.balance = initial_balance
        self.coins = {}

    def add_coin(self, name, quantity, value):
        cost = quantity * value
        # Check if have enough money
        if self.balance > cost:
            self.balance -= cost
            if name in self.coins:
                self.coins[name] += quantity
            else:
                self.coins[name] = quantity
            return True
        else:
            return False

    def remove_coin(self, name, quantity, value):
        if name in self.coins:
            if self.coins[name] >= quantity:
                cost = quantity * value
                self.balance += cost
                self.coins[name] -= quantity
                if self.coins[name] == 0:
                    del self.coins[name]
                return True
            else:
                return False
        else:
            return False

    def get_coin_quantity(self, name):
        if name in self.coins:
            return self.coins[name]
        else:
            return 0

    def get_balance(self):
        return self.balance
    
    def get_coins(self):
        return self.coins

## Define Strategy

In [4]:
class Strategy:
    # Simple moving average strategy

    def __init__(self, window_size):
        self.window_size = window_size
        self.price_history = []

    def update_price_history(self, price):
        self.price_history.append(price)
        if len(self.price_history) > self.window_size:
            self.price_history.pop(0)

    def generate_signals(self, observed_price):
        if len(self.price_history) < self.window_size:
            return 'HOLD'
        average_price = sum(self.price_history) / self.window_size
        if observed_price > average_price:
            return 'BUY'
        else:
            return 'SELL'

## Backtesting Strategy

### Fetch Data

In [5]:
def fetchData(client, symbol, interval, start_time, end_time):
    klines = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start_time, end_str=end_time)
    close_prices = [float(kline[4]) for kline in klines]
    timestamps = [datetime.fromtimestamp(kline[0] / 1000) for kline in klines]

    data = pd.DataFrame({'Timestamp': timestamps, 'Close Price': close_prices})

    return data

### Simulating

In [14]:
def simulation(data, wallet, strategy, symbol):
    data_balance = [None] * len(data)
    data_signal = [None] * len(data)

    coin_name = symbol.replace('USDT', '')

    for row in data.itertuples():
        data_index = row[0]
        close_price = row[2]

        strategy.update_price_history(close_price)
        signal = strategy.generate_signals(close_price)

        if signal == 'BUY':
            available_balance = wallet.get_balance()
            if available_balance > 1:
                quantity = available_balance / close_price
                if wallet.add_coin(coin_name, quantity, close_price):
                    data_signal[data_index] = 'BUY'

        elif signal == 'SELL':
            coin_pair = wallet.get_coins()
            if len(coin_pair) > 0:
                for coin_name_in_wallet, coin_qty_in_wallet in coin_pair.items():
                    if coin_name == coin_name_in_wallet:
                        if wallet.remove_coin(coin_name, coin_qty_in_wallet, close_price):
                            data_signal[data_index] = 'SELL'
                        break
        
        data_balance[data_index] = wallet.get_balance()

    # Force sell all coin
    coin_pair = wallet.get_coins()
    if len(coin_pair) > 0:
        for coin_name_in_wallet, coin_qty_in_wallet in coin_pair.items():
            if coin_name == coin_name_in_wallet:
                if wallet.remove_coin(coin_name, coin_qty_in_wallet, close_price):
                    data_signal[data_index] = 'SELL'
                    data_balance[-1] = wallet.get_balance()
                break

    data['Signal'] = data_signal
    data['Balance'] = data_balance
    return data

### Parameters

#### Input Parameters

In [73]:
interval = Client.KLINE_INTERVAL_1DAY  # Replace with the desired interval (e.g., KLINE_INTERVAL_1MINUTE, KLINE_INTERVAL_1HOUR)
start_time_train = int(datetime(2020,1,1,0,0).timestamp() * 1000)
start_time_train_plus1 = int(datetime(2020,1,2,0,0).timestamp() * 1000)
end_time_train = int(datetime(2021,12,31,0,0).timestamp() * 1000)

start_time_val = int(datetime(2022,1,1,0,0).timestamp() * 1000)
end_time_val_minus1 = int(datetime(2023,12,1,0,0).timestamp() * 1000)
end_time_val = int(datetime(2023,12,31,0,0).timestamp() * 1000)

initial_balance = 1000

#### Fetch Training data set from binance

In [74]:
ticker_infos = client.get_ticker()
symbols = [symbol['symbol'] for symbol in ticker_infos if symbol['symbol'].endswith('USDT')]

In [75]:
symbols_remove = []
for symbol in symbols:
    data_test = fetchData(client, symbol, interval, start_time_train, start_time_train_plus1)
    if data_test.empty:
        symbols_remove.append(symbol)
        continue
    data_test = fetchData(client, symbol, interval, end_time_val_minus1, end_time_val)
    if data_test.empty:
        symbols_remove.append(symbol)

for symbol in symbols_remove:
    symbols.remove(symbol)

## Results

### Optimizing

In [76]:
optimize_results = pd.DataFrame(columns=['symbol', 'profit margin', 'window size'])

for symbol in symbols:
    data_train = fetchData(client, symbol, interval, start_time_train, end_time_train)
    data_val   = fetchData(client, symbol, interval, start_time_val  , end_time_val)
    
    # Select window size

    max_profit = 0
    window_size_selected = 0
    for window_size in range(10,101):
        wallet         = Wallet(initial_balance=initial_balance)
        strategy       = Strategy(window_size=window_size)
        sim            = simulation(data_train, wallet, strategy, symbol)
        ending_balance = sim['Balance'].iloc[-1]
        if ending_balance > max_profit:
            window_size_selected = window_size
            max_profit = ending_balance
            
    # Evaluate profitability

    wallet = Wallet(initial_balance=initial_balance)
    strategy = Strategy(window_size=window_size_selected)
    sim = simulation(data_val, wallet, strategy, symbol)
    ending_balance = sim['Balance'].iloc[-1]
    profit_margin = ending_balance / initial_balance

    new_row = {'symbol': symbol, 'profit margin': profit_margin, 'window size': window_size_selected}

    optimize_results.loc[len(optimize_results)] = new_row

### Check

In [45]:
symbol = 'ETHUSDT'

data = fetchData(client, symbol, interval, start_time_val, end_time_val)

wallet = Wallet(initial_balance=1000)
strategy = Strategy(window_size=77)
sim = simulation(data, wallet, strategy, symbol)

In [50]:
ending_balance = sim['Balance'].iloc[-1]
profit_margin = ending_balance / initial_balance
profit_margin

np.float64(0.727280537730831)

In [78]:
optimize_results[optimize_results['profit margin']>1]

,symbol,profit margin,window size
5,QTUMUSDT,1.042153,25
6,ADAUSDT,1.487418,47
10,IOTAUSDT,1.366643,39
14,ETCUSDT,1.522486,71
15,ICXUSDT,1.073235,11
17,VETUSDT,1.208104,51
22,HOTUSDT,1.070777,27
23,ZILUSDT,1.895816,61
25,FETUSDT,6.586345,42
27,XMRUSDT,1.395111,36
